In [1]:
inputs = readdir("../graphs");

In [58]:
function parseData(fileName::ASCIIString, tabs::ASCIIString)
#     println(fileName)
    
    f = open(fileName)
    lines = readlines(f)
    close(f)
    
    results = Array{Array{Float64,1},1}(0)

    # parse the input text file to ASCIIString
    nr = parse(Int64, split(lines[2], ' ')[1])
    for i in 3:(3 + nr - 1)
        wholeLn = split(lines[i], [' ', '=', '\n'])
        ln = []
        for j in wholeLn
            if j != ""
                push!(ln, j)
            end
        end
        
        thisLine = []
        for j in 4:2:length(ln)
            push!(thisLine, parse(Float64, ln[j]))
        end
        
        push!(results, thisLine)
    end
    
    ans = []
    
    # get the different treshhold for convergence
    for conv in [1000,10,1e-1,1e-3,1e-5,1e-7]
        found = false
        for i in 1:nr
            if results[i][1] < conv
                push!(ans, tabs * string(conv) * " -> " * string(i))
                found = true
                break
            end
        end
        
        if found == false
            push!(ans, tabs * string(conv) * " -> " * "N/A")
        end
    end
    
    return ans
    
end

parseData (generic function with 1 method)

In [59]:
output = []

for inpt in inputs
    goodstuff = false
    if ((inpt[1] == 'k' && inpt[2] == 'e') || 
        (inpt[1] == 's' && inpt[2] == 'e') ||
        (inpt[1] == 'x' && inpt[2] == 'u'))
        goodstuff = true
    end
    
    if !goodstuff
        continue
    end
    
    push!(output, inpt)
    
    dir = "../graphs/" * inpt;
    allLogs = readdir(dir)
    
    julLogs = []
    for log in allLogs
        if search(log, "julia").stop != -1
            push!(julLogs, log)
        end
    end
#     println(dir, " ", julLogs)
    
    tabspace = "    "
    
    for log in julLogs
        if search(log, "default").stop != -1
            push!(output, tabspace * string(53))
            
            append!(output, parseData(dir * "/" * log, tabspace * tabspace))
        end
    end
        
    for p2 in 5:20
        for log in julLogs
            if search(log, string(2^p2)).stop != -1
                push!(output, tabspace * string(2^p2))
                
                append!(output, parseData(dir * "/" * log, tabspace * tabspace))
            end
        end
    end
    
end

In [60]:
f = open("../se_results_anorm.txt", "w")
for otpt in output
    println(f, otpt)
end
close(f)